# Static Explorative Data Analysis

In [1]:
from pathlib import Path
import sys
root_dir = Path().resolve().parent
sys.path.append(str(root_dir))



import rsdb.preprocess.data_preprocessing as data_preprocessing
import rsdb.features.featuring as featuring
import gzip
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import random
import warnings
from collections import Counter
import string
import folium
from folium.plugins import MarkerCluster
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None) # display whole column 

In [ ]:
print(Path.cwd().parent.parent)

In [ ]:
print("Current Working Directory:", Path.cwd().parent)

In [ ]:
url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/review-California_10.json.gz"
meta_url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/meta-California.json.gz"
# df = data_preprocessing.get_clean_review_data(url, meta_url)

df = data_preprocessing.get_single_chunk(url, meta_url)

## Check featuring categories

In [ ]:
temp = featuring.featuring_category(df, ['restaurant', 'park', 'store'])
x_temp = temp[['rating', 'isin_category_restaurant', 'isin_category_park', 'isin_category_store']]

x_temp.groupby(['isin_category_restaurant', 'isin_category_park', 'isin_category_store'])['rating'].mean()

In [ ]:

# def category_count(df):
#     """
#     Count the size of the 'category' array for each row in the dataframe
#     and add a new column 'category_count'.

#     Args:
#         df (pd.DataFrame): Input dataframe with a 'category' column containing lists.

#     Returns:
#         pd.DataFrame: The updated dataframe with a new 'category_count' column.
#     """
#     # Ensure 'category' column exists in the DataFrame
#     if 'category' not in df.columns:
#         raise ValueError("The DataFrame must contain a 'category' column.")
    
#     # Calculate the size of the category array for each row
#     df['category_count'] = df['category'].apply(len)
    
#     return df

# temp_x = category_count(df)
# df['rating'].corr(df['category_count'])


# df[['rating', "category_count"]]

## Check Feature: location on rating

In [38]:
def featuring_locations(df: pd.DataFrame, lon_bins=20, lat_bins=20) -> pd.DataFrame:
    """
    takes in a dataframe and divide longitude and latitude into equally
    distributed bins

    Args:
        df: input dataframe
        lon_bins: number of bins for longitude
        lat_bins: number of bins for latitude

    return: dataframe with bins encoded into categories

    """
    assert "longitude" in df.columns, "longitude not in the dataframe"
    assert "latitude" in df.columns, "latitude not in the dataframe"

    # Calculate bin edges for longitude and latitude
    lon_edges = np.linspace(df["longitude"].min(), df["longitude"].max(), lon_bins + 1)
    lat_edges = np.linspace(df["latitude"].min(), df["latitude"].max(), lat_bins + 1)

    lon_bins = pd.cut(
        df["longitude"], bins=lon_edges, labels=False, include_lowest=True
    )
    lat_bins = pd.cut(df["latitude"], bins=lat_edges, labels=False, include_lowest=True)


    return df.assign(lon_bin=lon_bins).assign(lat_bin=lat_bins)

    # lon_feature_df = pd.get_dummies(lon_bins, prefix="lon_bin", dtype=int)
    # lat_feature_df = pd.get_dummies(lat_bins, prefix="lat_bin", dtype=int)

    # return pd.concat([df, lon_feature_df, lat_feature_df], axis=1).drop(
    #     columns=["longitude", "latitude"]
    # )

In [ ]:
featuring_locations(df).groupby(['lon_bin', 'lat_bin'])['rating'].mean().sort_values()

In [28]:
def milliseconds_to_years(milliseconds: int) -> float:
    """
    turn milliseconds into years

    Args:
        milliseconds

    return: number that convert milliseconds into years
    """
    seconds = milliseconds / 1000
    minutes = seconds / 60
    hours = minutes / 60
    days = hours / 24
    years = days / 365.25  # Account for leap years
    return years

def featuring_review_counts(df: pd.DataFrame) -> pd.DataFrame:
    """
    takes in a dataframe and count the average review per year of each gmapid

    Args:
        pd: input dataframe

    return: dataframe with bins encoded into categories
    """
    assert "review_time(unix)" in df.columns, "no review time"
    assert "gmap_id" in df.columns, "needs location identifier"
    assert "num_of_reviews" in df.columns, "needs total review counts"

    latest_review_time = df["review_time(unix)"].max()
    location_earliest_review = df.groupby(["gmap_id"])["review_time(unix)"].min()
    location_duration_ms =  latest_review_time - location_earliest_review
    location_duration_yr = location_duration_ms.apply(milliseconds_to_years)
    location_duration_yr_reviws = df[["gmap_id", "num_of_reviews"]].merge(
        location_duration_yr, left_on="gmap_id", right_index=True
    )

    assert location_duration_yr_reviws.shape[0] == df.shape[0], "merging issue"

    return df.assign(
        **{'avg_review(per year)': location_duration_yr_reviws['num_of_reviews'] / location_duration_yr_reviws['review_time(unix)']}
    )

In [29]:
test = featuring_review_counts(df)

In [ ]:
test.columns

In [ ]:
test['avg_review(per year)'].corr(test['avg_rating'])

In [ ]:
df

In [5]:
from rsdb.features.featuring import *


# # Create the new columns
# df['closed_on_weekend'] = df['hours'].apply(lambda x: is_closed_on_weekend(x))
# df['total_hours'] = df['hours'].apply(calculate_total_hours)

test12 = featuring_hours(df)#['is_closed_on_weekend']

In [ ]:
featuring_engineering(df)

In [ ]:
test12['weekly_operating_hours'].corr(test12['rating'])

In [ ]:
# Weekly bins for item bias
data_query["time_bin"] = data_query["review_time(unix)"] // (
    7 * 24 * 3600
)  # same week in same bin

# Mean timestamp for user bias
user_mean_times = data_query.groupby("reviewer_id")["review_time(unix)"].mean()
data_query["user_mean_time"] = data_query["reviewer_id"].map(user_mean_times)


train_df = data_query.sample(frac=0.8, random_state=42)
test_df = data_query.drop(train_df.index)

# Calculate mean and std for normalization
time_mean, time_std = (
    train_df["review_time(unix)"].mean(),
    train_df["review_time(unix)"].std(),
)
user_mean_time_mean, user_mean_time_std = (
    train_df["user_mean_time"].mean(),
    train_df["user_mean_time"].std(),
)

# Normalize the training and test data
train_df["review_time(unix)"] = (train_df["review_time(unix)"] - time_mean) / time_std
test_df["review_time(unix)"] = (test_df["review_time(unix)"] - time_mean) / time_std
train_df["user_mean_time"] = (
    train_df["user_mean_time"] - user_mean_time_mean
) / user_mean_time_std
test_df["user_mean_time"] = (
    test_df["user_mean_time"] - user_mean_time_mean
) / user_mean_time_std


# Sort data by user and timestamp and reate sequences

# sort dataframe and shift the 


data = merged_clean.sort_values(by=["reviewer_id", "review_time(unix)"])
data["prev_item_id"] = data.groupby("reviewer_id")["gmap_id"].shift(1)
data = data.dropna(subset=["prev_item_id"])

In [ ]:


def find_top_words_by_period_sampled(df, column, time_column, periods, sample_size=10000):
    """
    Finds the top 10 most common words for each time period using a sampled subset of the data.

    Args:
        df (DataFrame): Input DataFrame.
        column (str): Column containing the text data.
        time_column (str): Column containing Unix time (in milliseconds).
        periods (list of tuples): List of (start, end) time ranges for filtering.
        sample_size (int): Number of rows to sample for each period. Default is 10,000.

    Returns:
        dict: Dictionary with periods as keys and top 10 words as values.
    """
    # Initialize a dictionary to store results
    top_words_by_period = {}

    # Predefine stopwords and punctuation
    stop_words = set([
        'the', 'and', 'a', 'to', 'of', 'in', 'for', 'on', 'with', 'at', 'it', 'this', 'is', 'from', 'by', 'as', 'that', 'which', 'or', 'an'
        
    ])
    punctuations = set(string.punctuation)

    for start, end in periods:
        # Filter DataFrame for the time period
        period_df = df[(df[time_column] >= start) & (df[time_column] < end)]

        # Sample a subset of the data
        sampled_df = period_df.sample(n=min(sample_size, len(period_df)), random_state=42)

        # Combine all text into one string from the sampled data
        all_text = " ".join(sampled_df[column].dropna().astype(str).tolist())

        # Tokenize words, convert to lowercase, and remove stopwords/punctuation
        words = [
            word.lower() for word in all_text.split()
            if word.lower() not in stop_words and word not in punctuations
        ]

        # Count word frequencies and get the top 10
        word_counts = Counter(words)
        top_words = word_counts.most_common(10)

        # Store results for this period
        top_words_by_period[f"{start}-{end}"] = top_words

    return top_words_by_period


In [ ]:
# Define time periods (Unix time in milliseconds)
periods = [
    (1104537600 * 1000, 1262304000 * 1000),  # 2005-2010
    (1262304000 * 1000, 1420070400 * 1000),  # 2010-2015
    (1420070400 * 1000, 1577836800 * 1000),  # 2015-2020
    (1577836800 * 1000, 1704067200 * 1000),  # 2020-2024
]

# Find top words for each period, with sampling
top_words = find_top_words_by_period_sampled(df, column='text', time_column='review_time(unix)', periods=periods, sample_size=10000)

# Print results
for period, words in top_words.items():
    print(f"Top words for {period}:")
    for word, count in words:
        print(f"{word}: {count}")
    print()


In [ ]:


def average_rating_maps(df, time_column, rating_column, periods, sample_size=100000, location_column="latitude, longitude"):
    """
    Generates maps showing the average rating of all locations over time.

    Args:
        df (DataFrame): The DataFrame containing the review data.
        time_column (str): Column containing Unix time (in milliseconds).
        rating_column (str): Column containing the review ratings.
        periods (list of tuples): List of (start, end) time ranges for filtering.
        sample_size (int): Number of rows to sample from the entire DataFrame. Default is 100,000.
        location_column (str): Column for the location. Assumes a tuple for (latitude, longitude).
        
    Returns:
        List of Folium Maps for each time period.
    """
    # Sample the dataframe once
    sampled_df = df.sample(n=sample_size, random_state=42)

    # List to store the maps
    maps = []
    
    for i, (start, end) in enumerate(periods):
        # Filter the data for the current period
        period_df = sampled_df[(sampled_df[time_column] >= start) & (sampled_df[time_column] < end)]

        # Group by location (latitude and longitude) and calculate the average rating for each location
        avg_ratings_per_location = period_df.groupby(['latitude', 'longitude'])[rating_column].mean().reset_index()

        # Create a Folium map centered around a mean location
        avg_lat = avg_ratings_per_location['latitude'].mean()
        avg_lon = avg_ratings_per_location['longitude'].mean()
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=6)

        # Create a marker cluster
        marker_cluster = MarkerCluster().add_to(m)

        # Add a marker for each location with average rating
        for _, row in avg_ratings_per_location.iterrows():
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=5,
                popup=f"Avg Rating: {row[rating_column]:.2f}",
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.6
            ).add_to(marker_cluster)
        
        # Title for the map based on the period
        period_label = f"Average Rating (from {pd.to_datetime(start, unit='ms').year} to {pd.to_datetime(end, unit='ms').year})"
        maps.append(m)
        m.save(f"average_rating_map_{i}.html")  # Save map as HTML file
    
    return maps

# Define time periods (Unix time in milliseconds)
periods = [
    (1104537600 * 1000, 1262304000 * 1000),  # 2005-2010
    (1262304000 * 1000, 1420070400 * 1000),  # 2010-2015
    (1420070400 * 1000, 1577836800 * 1000),  # 2015-2020
    (1577836800 * 1000, 1704067200 * 1000),  # 2020-2024
]

# Call the function to generate the maps
maps = average_rating_maps(df, time_column='review_time(unix)', rating_column='avg_rating', periods=periods, sample_size=100000)


In [ ]:


# Sample 1000 entries from the dataframe
df_sample = df.sample(n=1000, random_state=42)

# Extract latitude, longitude, and category for each entry
df_sample['latitude'] = df_sample['latitude']
df_sample['longitude'] = df_sample['longitude']
df_sample['category'] = df_sample['category']

# Create a function to get the most popular category for each location
def get_most_popular_category(location_df):
    # Count the occurrences of each category for a given location
    category_counts = location_df['category'].explode().value_counts()
    if not category_counts.empty:
        return category_counts.idxmax()  # Get the most frequent category
    return None

# Group by location and get the most popular category for each location
location_categories = df_sample.groupby(['latitude', 'longitude']).apply(get_most_popular_category).reset_index(name='most_popular_category')

# Create the map centered around an average location (or any central location)
map_center = [df_sample['latitude'].mean(), df_sample['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

# Add markers for each location with the most popular category
for idx, row in location_categories.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Category: {row['most_popular_category']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(mymap)

# Save the map to an HTML file and display it
mymap.save("most_popular_categories_map.html")

# To display the map inline in Jupyter or similar environments, you can use:
# mymap


In [ ]:
# Sample 10000 entries from the dataframe
sample_size = 10000
df_sample = df.sample(n=sample_size, random_state=42)

# Define latitude and longitude bounds
lat_min, lat_max = 32.5, 42
long_min, long_max = -124.4, -114.13

# Filter out rows where latitude and longitude are outside the bounds
df_sample_filtered = df_sample[
    (df_sample['latitude'] >= lat_min) & (df_sample['latitude'] <= lat_max) & 
    (df_sample['longitude'] >= long_min) & (df_sample['longitude'] <= long_max)
]

# Create a function to get the most popular category for each location
def get_most_popular_category(location_df):
    # Count the occurrences of each category for a given location
    category_counts = location_df['category'].explode().value_counts()
    if not category_counts.empty:
        return category_counts.idxmax()  # Get the most frequent category
    return None

# Group by location and get the most popular category for each location
location_categories = df_sample_filtered.groupby(['latitude', 'longitude']).apply(get_most_popular_category).reset_index(name='most_popular_category')

# Merge the most_popular_category column back into df_sample_filtered
df_sample_filtered = pd.merge(df_sample_filtered, location_categories, on=['latitude', 'longitude'], how='left')

# Get the top 10 most popular categories
top_categories = df_sample_filtered['most_popular_category'].value_counts().nlargest(10).index

# Filter the dataframe to include only the top 10 categories
df_sample_filtered = df_sample_filtered[df_sample_filtered['most_popular_category'].isin(top_categories)]

# Map each category to a color
category_colors = {category: color for category, color in zip(df_sample_filtered['most_popular_category'].unique(), sns.color_palette("Set2", len(df_sample_filtered['most_popular_category'].unique())))}

# Create the plot (only one subplot for the most popular category distribution)
plt.figure(figsize=(8, 6))

# Scatterplot for Most Popular Category Distribution
sns.scatterplot(
    data=df_sample_filtered,
    x="longitude", 
    y="latitude", 
    hue="most_popular_category", 
    palette=category_colors,
    size="num_of_reviews", 
    sizes=(20, 200), 
    alpha=0.7
)

plt.title('Top 10 Most Popular Category Distribution (Sample)', fontsize=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Move the legend outside the plot
plt.legend(title="Category", loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:

# Sample 10000 entries from the dataframe
sample_size = 10000
df_sample = df.sample(n=sample_size, random_state=42)

# Define latitude and longitude bounds
lat_min, lat_max = 32.5, 42
long_min, long_max = -124.4, -114.13

# Filter out rows where latitude and longitude are outside the bounds
df_sample_filtered = df_sample[
    (df_sample['latitude'] >= lat_min) & (df_sample['latitude'] <= lat_max) & 
    (df_sample['longitude'] >= long_min) & (df_sample['longitude'] <= long_max)
]

# Create a function to get the most popular category for each location
def get_most_popular_category(location_df):
    # Count the occurrences of each category for a given location
    category_counts = location_df['category'].explode().value_counts()
    if not category_counts.empty:
        return category_counts.idxmax()  # Get the most frequent category
    return None

# Group by location and get the most popular category for each location
location_categories = df_sample_filtered.groupby(['latitude', 'longitude']).apply(get_most_popular_category).reset_index(name='most_popular_category')

# Merge the most_popular_category column back into df_sample_filtered
df_sample_filtered = pd.merge(df_sample_filtered, location_categories, on=['latitude', 'longitude'], how='left')

# Get the top 20 most popular categories
top_categories = df_sample_filtered['most_popular_category'].value_counts().nlargest(20).index

# Filter the dataframe to include only the top 20 categories
df_sample_filtered = df_sample_filtered[df_sample_filtered['most_popular_category'].isin(top_categories)]

# Map each category to a color
category_colors = {category: color for category, color in zip(df_sample_filtered['most_popular_category'].unique(), sns.color_palette("Set2", len(df_sample_filtered['most_popular_category'].unique())))}

# Create the plot (only one subplot for the most popular category distribution)
plt.figure(figsize=(8, 6))

# Scatterplot for Most Popular Category Distribution
sns.scatterplot(
    data=df_sample_filtered,
    x="longitude", 
    y="latitude", 
    hue="most_popular_category", 
    palette=category_colors,
    alpha=0.7
)

plt.title('Top 20 Most Popular Category Distribution (Sample)', fontsize=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Move the legend outside the plot
plt.legend(title="Category", loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Sample 50000 entries from the dataframe
sample_size = 50000
df_sample = df.sample(n=sample_size, random_state=42)

# Define latitude and longitude bounds
lat_min, lat_max = 32.5, 42
long_min, long_max = -124.4, -114.13

# Filter out rows where latitude and longitude are outside the bounds
df_sample_filtered = df_sample[
    (df_sample['latitude'] >= lat_min) & (df_sample['latitude'] <= lat_max) & 
    (df_sample['longitude'] >= long_min) & (df_sample['longitude'] <= long_max)
]

# Filter the dataframe for the selected categories
categories_of_interest = ['Shopping mall', 'Grocery store', 'Restaurant', 'Movie theater']
df_sample_filtered = df_sample_filtered[df_sample_filtered['category'].apply(lambda x: any(cat in categories_of_interest for cat in x))]

# Explode the categories column to work with individual categories
df_sample_filtered_exploded = df_sample_filtered.explode('category')

# Keep only the rows where the category is in the list of interest
df_sample_filtered_exploded = df_sample_filtered_exploded[df_sample_filtered_exploded['category'].isin(categories_of_interest)]

# Create a color palette for the categories
category_colors = {
    'Shopping mall': 'blue',
    'Grocery store': 'green',
    'Restaurant': 'red',
    'Movie theater': 'purple'
}

# Create the plot
plt.figure(figsize=(10, 6))

# Scatterplot for selected categories
sns.scatterplot(
    data=df_sample_filtered_exploded,
    x="longitude", 
    y="latitude", 
    hue="category", 
    palette=category_colors,
    alpha=0.7
)

# Set the title and labels
plt.title('Distribution of Shopping Mall, Grocery Store, Restaurant, and Movie Theater', fontsize=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Move the legend outside the plot
plt.legend(title="Category", loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Sample 1000 entries from the dataframe (adjust the sample size as needed)
sample_size = 300000
df_sample = df.sample(n=sample_size, random_state=42)

# Assuming you have a 'review_time' column or similar that contains datetime or Unix timestamps
# If the column is a Unix timestamp (in milliseconds), we first need to convert it to a datetime object
df_sample['review_time'] = pd.to_datetime(df_sample['review_time(unix)'], unit='ms')

# Extract the year from the 'review_time' column
df_sample['year'] = df_sample['review_time'].dt.year

# Filter the dataset to include only the specific business types we're interested in
business_types = ['Restaurant', 'Shopping mall', 'Park', 'Grocery store', 'Movie theater']
df_sample_filtered = df_sample[df_sample['category'].apply(lambda x: any(business in x for business in business_types))]

# Create a new column that maps the specific business types
def map_business_type(categories):
    for business in business_types:
        if business in categories:
            return business
    return None  # In case no category matches

df_sample_filtered['business_type'] = df_sample_filtered['category'].apply(map_business_type)

# Group by year and business type, then count the number of businesses for each
businesses_per_year_type = df_sample_filtered.groupby(['year', 'business_type']).size().reset_index(name='count')

# Create the line plot
plt.figure(figsize=(10, 4))

# Create a line plot for each business type
sns.lineplot(x='year', y='count', hue='business_type', data=businesses_per_year_type, marker='o')

# Set the title and labels
plt.title('Number of Businesses by Type Over Time', fontsize=14)
plt.xlabel('Year')
plt.ylabel('Number of Businesses')

# Display the legend outside the graph
plt.legend(title="Business Type", bbox_to_anchor=(1.05, 1), loc='upper left')

# Adjust x-axis to display every year, not just specific intervals
plt.xticks(businesses_per_year_type['year'].unique(), rotation=45)  # Ensure each year is shown

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Step 1: Find unique keys in the dictionaries within the MISC column
unique_keys = set()

# Iterate through each dictionary in the 'MISC' column and collect the keys
for misc_dict in df['MISC']:
    if isinstance(misc_dict, dict):  # Ensure the value is a dictionary
        unique_keys.update(misc_dict.keys())

# Print unique keys
print("Unique keys in MISC:", unique_keys)

# Step 2: Find unique values for each key (handling lists as values)
unique_values = {}

# Iterate through each dictionary again to get unique values for each key
for key in unique_keys:
    # Collect unique values for the current key
    values_for_key = set()
    for misc_dict in df['MISC']:
        if isinstance(misc_dict, dict) and key in misc_dict:
            # Add each item from the list to the set for unique values
            values_for_key.update(misc_dict[key])
    unique_values[key] = values_for_key

# Print the number of unique values for each key
for key, values in unique_values.items():
    print(f"Key: {key} has {len(values)} unique values: {values}")


In [ ]:
# Create a separate DataFrame for one-hot encoded columns
one_hot_encoded_df = pd.DataFrame(index=df.index)  # Use the same index as the original df

# Iterate through each row in the dataframe to check for key-value presence
for index, row in df.iterrows():
    if isinstance(row['MISC'], dict):
        for key, values in unique_values.items():
            # For each key, iterate through the possible values
            for value in values:
                # Create a new column name based on key-value
                column_name = f"{key}_{value}"
                
                # Check if the value exists for the key in the dictionary
                if key in row['MISC'] and value in row['MISC'][key]:
                    # If present, set the column to 1
                    one_hot_encoded_df.at[index, column_name] = 1
                else:
                    # If not present, set the column to 0
                    one_hot_encoded_df.at[index, column_name] = 0

# Step 3: Final clean-up: Replace NaN with 0 (since new columns may have NaN values initially)
one_hot_encoded_df = one_hot_encoded_df.fillna(0)

# Now, one_hot_encoded_df contains the one-hot encoded columns
print(one_hot_encoded_df.head())  # See the result